# Robust Phase Estimation: one and two qubit examples

In [ ]:
import numpy as np
from numpy import pi
from forest_benchmarking import rpe
from pyquil import get_qc, Program
from pyquil.gates import RZ, RX, RY, I

# get a qauntum computer object. Here we use a noisy qvm. 
qc = get_qc("9q-square", as_qvm=True, noisy=True)

## Generate a Dataframe to Estimate the Phase of RZ(angle, qubit)

In [ ]:
# we start with determination of an angle of rotation about the Z axis
rz_angle = 2 # we will use an ideal gate with phase of 2 radians
qubit = 0
rotation = RZ(rz_angle, qubit) 
# the rotation is about the Z axis; the eigenvectors are the computational basis states
# therefore the change of basis is trivially the identity. 
change_of_basis = I(qubit) 
rz_experiment = rpe.generate_rpe_experiment(rotation, change_of_basis)

## Acquire the Data

In [ ]:
rz_results = rpe.acquire_rpe_data(qc, rz_experiment)
# we get back a copy of the experiment with a new "Resutls" column with raw shot data.
rz_results["Results"]

## Get the estimate of the phase

We hope that the estimated phase is close to our choice for rz_angle=2

In [ ]:
rpe.robust_phase_estimate(rz_results)

## Now let's try a rotation around the +X axis

In [ ]:
rx_angle = pi # radians; only x gates with phases in {-pi, -pi/2, pi/2, pi} are allowed
qubit = 1 # let's use a different qubit
rotation = RX(rx_angle, qubit)
# the rotation has eigenvectors |+> and |->; 
# change of basis needs to rotate |0> to the plus state and |1> to the minus state
change_of_basis = RY(pi/2, qubit)
rx_experiment = rpe.generate_rpe_experiment(rotation, change_of_basis=change_of_basis)

In [ ]:
rx_experiment

In [ ]:
rx_results = rpe.acquire_rpe_data(qc, rx_experiment)
# we hope this is close to rx_angle = pi
rpe.robust_phase_estimate(rx_results)

## Hadamard-like rotation

We can do any rotation which is expressed in native gates (or, more generally, using gates included in basic_compile() in forest_benchmarking.compilation). There are helper functions to help determine the proper change of basis required to do such an experiment. Here, we use the fact that a Hadamard interchanges X and Z, and so must constitute a rotation about the axis (pi/4, 0) on the Bloch sphere. From this, we find the eigenvectors of an arbitrary rotation about this axis, and use those to find the change of basis matrix. Note that the change of basis need not be a program or gate; if a matrix is supplied then it will be translated to a program using a qc object's compiler. This can be done manually with a call to add_programs_to_rpe_dataframe(qc, expt) or automatically when acquire_rpe_data is called.

In [ ]:
# create a program implementing the rotation of a given angle about the "Hadamard axis" 
rh_angle = 1.5  # radians
qubit = 0
RH = Program(RY(-pi / 4, qubit)).inst(RZ(rh_angle, qubit)).inst(RY(pi / 4, qubit))

# get the eigenvectors knowing the axis of rotation is pi/4, 0
evecs = rpe.bloch_rotation_to_eigenvectors(pi / 4, 0)

# get a ndarray representing the change of basis transformation
cob = rpe.get_change_of_basis_from_eigvecs(evecs)

# create an experiment as usual
rh_experiment = rpe.generate_rpe_experiment(RH, cob)

# Optionally perform a manual step to see the full program before running. Note a qc object is required.
# This can be skipped, and a program column will instead be added when acquire_rpe_data is called
rh_experiment = rpe.add_programs_to_rpe_dataframe(qc, rh_experiment)

# get the results per usual
rh_results = rpe.acquire_rpe_data(qc, rh_experiment, multiplicative_factor=2)

# the result should be close to our input rh_angle=1.5
rpe.robust_phase_estimate(rh_results)

## We can also group multiple experiments to run simultaneously

This will lead to shorter data acquisition times on a QPU. Only programs which act on disjoint sets of qubits will be run simultaneously. The group of experiments run together is recorded in each experiment under a "Simultaneous Group" column

In [ ]:
experiments = (rz_experiment, rx_experiment, rh_experiment)

In [ ]:
experiments_results = rpe.acquire_rpe_data(qc, experiments)

We should find that the experiments rz and rx, at indexes 0 and 1 in our list, have been grouped together. Meanwhile the rh experiment at index 2 had to be run after the first group.

In [ ]:
for expt in experiments_results:
    print(expt["Simultaneous Group"][0])

Iterate through the group and check that the estimates agree with previous results

In [ ]:
for expt in experiments_results:
    # we expect 2, pi, and 1.5 
    print(rpe.robust_phase_estimate(expt))

## For a particular experiment we can compute a predicted upper bound on the point estimate variance

In [ ]:
var = rpe.get_variance_upper_bound(rh_experiment)
# or use the actual number of shots that we took, which was twice the standard
var_with_adhoc_factor = rpe.get_variance_upper_bound(rh_results)

# we used a multiplicative factor of two to get rh_results
assert var_with_adhoc_factor == rpe.get_variance_upper_bound(rh_experiment, multiplicative_factor=2)

estimate = rpe.robust_phase_estimate(rh_results)

# difference between obs and actual should be less than predicted error 
print(np.abs(estimate - rh_angle), ' <? ', np.sqrt(var))

# difference should further be less than the error predicted given that we doubled the number of samples
print(np.abs(estimate - rh_angle), ' <? ', np.sqrt(var_with_adhoc_factor))

## We can visualize the rotation at each depth by plotting x and y expectations

This works best for small angles where there is less overlap

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

angle = pi/16
num_depths = 6

expt = rpe.generate_rpe_experiment(RZ(angle,0), I(0), num_depths=num_depths)
expt = rpe.acquire_rpe_data(qc, expt, multiplicative_factor = 100, additive_error = .1)
observed = rpe.robust_phase_estimate(expt)
print("Expected: ", angle)
print("Observed: ", observed)

expected = [(1.0, angle*2**j) for j in range(num_depths)]
ax = rpe.plot_rpe_iterations(expt, expected)
plt.show()

## We can also estimate the relative phases between eigenvectors of multi-qubit rotations

Note that for a particular 2q gate there are only three free eigenvalues; the fourth is determined by the special unitary condition. If we let  
$$\text{rotation } = \exp{(\pi i \text{ diag}(\phi_0, \phi_1, \phi_2, \phi_3))}$$ 
then the special unitary condition is $$\sum_j\phi_j = 0$$ 
Our experiment will return estimates
$$ [(\phi_2-\phi_0), (\phi_3-\phi_1), (\phi_1-\phi_0), (\phi_3-\phi_2) ]$$
from which each individual phases can be determined (indeed the system is over-determined).  

In the example below we demonstrate this procedure for our native CZ gate. The ideal gate has phases
$$(\phi_0=0, \phi_1=0, \phi_2=0, \phi_3=\pi)$$ 
To enforce special unitarity we ought to factor out an overall phase; let us rather note that the ideal relative phases in the order listed about are given by 
$$ [0, \pi, 0, \pi]$$

In [ ]:
from pyquil.gates import CZ

rotation = CZ(0, 1)
cob = Program([I(0),I(1)])  # CZ is diagonal in computational basis, i.e. no need to change basis
cz_expt = rpe.generate_rpe_experiment(rotation, cob, num_depths=7)
cz_expt = rpe.acquire_rpe_data(qc, cz_expt, multiplicative_factor = 50.0, additive_error=.1)
results = rpe.robust_phase_estimate(cz_expt)
# we hope to match the ideal results (0, pi, 0, pi)
print(results)

We can also alter the experiment by indicating a preparation-and-post-selection state for some of our qubits. In this type of experiment we prepare the superposition between only a subset of the eigenvectors and so estimate only a subset of the possible relative phases. In the two qubit case below we specify that qubit 0 be prepared in the one state and that we throw out any results where qubit 0 is not measured in this state. Meanwhile, qubit 1 is still prepared in the |+> state. The preparation state is thus the superposition of eigenvectors indexed 2 and 3, and we correspondingly measure the relative phase $$\phi_3 - \phi_2  = \pi$$ in the ideal case

In [ ]:
cz_single_phase = rpe.generate_rpe_experiment(rotation, cob, num_depths=7, prepare_and_post_select={0:1})
cz_single_phase = rpe.acquire_rpe_data(qc, cz_single_phase, multiplicative_factor = 50.0)
single_result = rpe.robust_phase_estimate(cz_single_phase)
# we hope the result is close to pi
print(single_result)

## Characterizing a universal 1q gateset with approximately orthogonal rotation axes (using simulated artificially imperfect gates)

In [ ]:
from pyquil import Program
from pyquil.quil import DefGate, Gate
from forest_benchmarking.utils import sigma_x, sigma_y, sigma_z
"""
Procedure and notation follows Sections III A, B, and C in 
[RPE]  Robust Calibration of a Universal Single-Qubit Gate-Set via Robust Phase Estimation
            Kimmel et al., Phys. Rev. A 92, 062315 (2015)
            https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.062315
            arXiv:1502.02677
"""    
q = 0  # pick a qubit

pauli_vector = np.array([sigma_x, sigma_y, sigma_z])

alpha = .01
epsilon = .02
theta = .5

# Section III A of [RPE]
gate1 = RZ(pi/2 * (1 + alpha), q) # assume some small over-rotation by fraction alpha

# let gate 2 be RX(pi/4) with over-rotation by fraction epsilon,
# and with a slight over-tilt of rotation axis by theta in X-Z plane
rx_angle = pi/4 * (1 + epsilon)
axis_unit_vector = np.array([np.cos(theta), 0, -np.sin(theta)])
mtrx = np.add(np.cos(rx_angle / 2) * np.eye(2), 
              - 1j * np.sin(rx_angle / 2) * np.tensordot(axis_unit_vector, pauli_vector, axes=1))
# Section III B of [RPE]

# get Quil definition for simulated imperfect gate
definition = DefGate('ImperfectRX', mtrx)
# get gate constructor
IRX = definition.get_constructor()
# set gate as program with definition and instruction, compiled into native gateset
gate2 = qc.compiler.quil_to_native_quil(Program([definition, IRX(q)]))
gate2 = Program([inst for inst in gate2 if isinstance(inst, Gate)])

# Section III B of [RPE], eq. III.3
# construct the program used to estimate theta
half = Program(gate1)
for _ in range(4):
    half.inst(IRX(q))
half.inst(gate1)
# compile into native gateset 
U_theta =  qc.compiler.quil_to_native_quil(Program([definition, half, half]))
U_theta = Program([inst for inst in U_theta if isinstance(inst, Gate)])


results = []
gates = [gate1, gate2, U_theta]
cobs = [I(q), RY(pi/2, q), RY(pi/2, q)]
for gate, cob in zip(gates, cobs):
    expt = rpe.generate_rpe_experiment(gate, cob)
    expt = rpe.acquire_rpe_data(qc, expt, multiplicative_factor = 50.0, additive_error=.15)
    result = rpe.robust_phase_estimate(expt)
    results += [result]
    
print("Expected Alpha: " + str(alpha))
print("Estimated Alpha: " + str(results[0]/(pi/2) - 1))
print()
print("Expected Epsilon: " + str(epsilon))
epsilon_est = results[1]/(pi/4) - 1
print("Estimated Epsilon: " + str(epsilon_est))
print()
print("Expected Theta: " + str(theta))
print("Estimated Theta: " + str(np.sin(results[2]/2)/(2*np.cos(epsilon_est * pi/2))))

## Example with Different Noise Model

In [ ]:
from pandas import Series
from pyquil.noise import damping_after_dephasing
from pyquil.quil import Measurement
qc = get_qc("9q-square", as_qvm=True, noisy=False)

def add_damping_dephasing_noise(prog, T1, T2, gate_time):
    p = Program()
    p.defgate("noise", np.eye(2))
    p.define_noisy_gate("noise", [0], damping_after_dephasing(T1, T2, gate_time))
    for elem in prog:
        p.inst(elem)
        if isinstance(elem, Measurement):
            continue  # skip measurement
        p.inst(("noise", 0))
    return p

def add_noise_to_experiments(df, t1, t2, p00, p11):
    df_copy = df.copy()
    gate_time = 200 * 10 ** (-9)
    df_copy["Program"] = Series([
        add_damping_dephasing_noise(prog, t1, t2, gate_time).define_noisy_readout(0, p00, p11)
        for prog in df_copy["Program"].values])
    return df_copy

angle = 1
RH = Program(RY(-pi / 4, 0)).inst(RZ(angle, 0)).inst(RY(pi / 4, 0))
evecs = rpe.bloch_rotation_to_eigenvectors(pi / 4, 0)
cob = rpe.get_change_of_basis_from_eigvecs(evecs)
expt = rpe.generate_rpe_experiment(RH, cob, num_depths=7)
expt = rpe.add_programs_to_rpe_dataframe(qc, expt)

# add noise to experiment with desired parameters
expt_with_noist = add_noise_to_experiments(expt, 25 * 10 ** (-6.), 20 * 10 ** (-6.), .92, .87)

expt_damp_dephase = rpe.acquire_rpe_data(qc, expt, multiplicative_factor=5., additive_error=.15)

You can also change the noise model of the qvm directly

In [ ]:
from pyquil.device import gates_in_isa
from pyquil.noise import decoherence_noise_with_asymmetric_ro, _decoherence_noise_model
# noise_model = decoherence_noise_with_asymmetric_ro(gates=gates_in_isa(qc.device.get_isa()), p00=0.92, p11=.87)
T1=20e-6
T2=10e-6
noise_model = _decoherence_noise_model(gates=gates_in_isa(qc.device.get_isa()), T1=T1, T2=T2, ro_fidelity=1.)

qc = get_qc("9q-square", as_qvm=True, noisy=False)
qc.qam.noise_model = noise_model

expt_decoherence = rpe.acquire_rpe_data(qc, expt, multiplicative_factor=5., additive_error=.15)

this is a simple way to manually add readout error after collecting results

In [ ]:
from numpy import random
def apply_readout_error(results, p00, p11):
    new_results = []
    for res in results:
        corrupted = []
        for bit in res:
            error = 0
            if bit == 0:
                if random.random() > p00:
                    error = 1
            else:
                if random.random() > p11:
                    error = 1
            
            corrupted.append((bit + error) % 2)
        new_results.append(corrupted)
    return np.array(new_results)

replace results with corrupted results exhibiting readout error

In [ ]:
from pandas import Series
expt_decoherence["Results"] = Series([apply_readout_error(res, .95, .85) for res in expt_decoherence["Results"]])

In [ ]:
damp_dephase_phase = rpe.robust_phase_estimate(expt_damp_dephase)
decoherence_phase = rpe.robust_phase_estimate(expt_decoherence)
print(damp_dephase_phase)
print(decoherence_phase)